In [114]:
import pandas as pd
from datetime import datetime, timedelta

import zipfile
from shapely.geometry import Point

import pyproj
import math

from collections import defaultdict

In [97]:
df = pd.read_excel('Report_JR14_Vehicle History Report - data.xlsx',skiprows=10)

In [98]:
'''
    Create Trips from the GPS Data
    1. 
'''
df.columns

Index(['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.', 'Rego',
       'Latitude', 'Longitude', 'Direction', 'Unnamed: 10', 'Speed', 'RawGPS',
       'Status', 'Flags', 'User Defined', 'Input', 'Output', 'Light', 'Button',
       'Max', 'Average', 'Spd Acc', 'Samples', 'Map Ref', 'Suburb', 'Distance',
       'Position', 'Eng Cool Temp', 'Eng Oil Temp', 'OilPressure',
       'ECM Economy', 'Trip', 'Total Used', 'RPM', 'Tot Hour', 'Brake Hits',
       'Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ', 'Position.1', '  1  ',
       '  2', '  3  ', '  4  ', '  5  ', '  6  ', '  7  ', '  8  ', '  9  ',
       '  10  ', 'Volts', 'Seconds', 'Speed.1', 'Zone 1', 'Zone 2', 'Zone 3',
       'Zone 4', 'Zone 1.1', 'Zone 2.1', 'Zone 3.1', 'Zone 4.1', '  1  .1',
       '  2  ', '  3  .1', '  4  .1', '  5  .1', 'Vehicle', 'Auxiliary',
       '  1  .2', '  2  .1', '  3  .2', '  4  .2', '  5  .2', 'Mass', 'Limit',
       'HDOP', 'Satellites', 'Temperature', 'From Flash', 'Offset Time',
       'Offset'

# Preprocessing
### Redundant. Do not run. DF gets altered

In [ ]:
'''
    Data Processing:

    Alpha: Check if Odometer Offset exists - Current logic searches for text Odometer Offset in Suburb. Best is to look in 'Offset' Column and get unique values
        Y: process add previous rows odo reading, add offset -- STATUS 1 ***OR***
        Y: filter the rows from the next day -- STATUS 2
        N: proceed usually. 
    1. Filter required columns out of overall dataset
    2. Concat Lat+Long into coords for each row
'''
# def manipulate_Odometer_Offsets(df,status):

#     odometer_offsets = df[df['Reason'].str.contains("Odometer Offset")].index.tolist()
    
#     # Basic Checks
#     # print(odometer_offsets)
#     if len(odometer_offsets) != 0 or status == 3:
#         return df
#     if status == 1:
#         # Write logic for taking previous rows and add the offset value
#         start = 0
#         for offset in range(len(odometer_offsets)):
#             end = odometer_offsets[offset]
#             value = df.loc[end]['Odometer']-df.loc[end-1]['Odometer']
#             df.loc[start:end]['Odometer'] += value
#             start = end
#         return df
#     elif status == 2:
#         date = None
#         for offset in range(len(odometer_offsets)):
#             date = pd.to_datetime(df.loc[odometer_offsets[offset]]['GPS']).date()
#             date = date + timedelta(days = 1)
#         return df[df['GPS'].dt.date >= date]
    
# Donot run odometet manipulator right now!
# df1 =manipulate_Odometer_Offsets(df,2)
def all_preprocessors(df):
    df['Date'] = pd.to_datetime(df['GPS'],format="%Y-%m-%d").dt.date
    df = df[['GPS', 'Device', 'Server', 'Reason', 'Driver', 'Desc.','Speed', 'Rego','Latitude', 'Longitude', 'Direction','Odometer', 'MaxF', 'MaxB', 'MaxLR', 'MaxZ','Offset Time','Offset']]
    df.loc[:, 'coords'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)
    return df
    # return manipulate_Odometer_Offsets(df,2)

# df = all_preprocessors(df)
# df = manipulate_Odometer_Offsets(df,1)

In [48]:
print(pd.to_datetime(df['GPS'].min()).strftime('%Y%m%d'))
print(df['Reason'].unique())

20230401
['Status Report' 'Diagnostic' 'Ignition On' 'Dep. Way Point'
 'Arr. Way Point' 'MDT not in use' 'MDVR Error - Hard Disk Status'
 'Ignition Off' 'MDVR Status' 'External Power Off' 'External Power On'
 'Logon' 'Break Start' 'Break End' 'Logoff' 'Unit Power Up'
 'GPS Settings Confirmed' 'Diagnostic Level' 'Odometer Offset'
 'Driver Missed Break Start' 'High G-Forces' 'MDVR Event'
 'G-Force Sustained Start' 'Position Report' 'G-Force Sustained End'
 'MDT not logged in' 'MDVR Diagnostic Error - Connection'
 'MDVR Diagnostic Error - Camera Connection' 'GForce Calibration - Start'
 'GForce Calibration - Stage 1 Complete' 'GForce Calibration - Finished'
 'MDVR Request' 'Driver Finished Break Early'
 'Gazetted Overspeed Zone 1 - Start (GPS)'
 'Gazetted Overspeed Zone 1 - End (GPS)'
 'Gazetted Overspeed Zone 1 - Tolerance Breached (GPS)']


In [115]:
depot = [-27.426770, 153.150956]
proj_wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')

def check_buffer(point1,point2):
    # Convert Shapely Point objects to (longitude, latitude) tuples
    lon1, lat1 = point1.x, point1.y
    lon2, lat2 = point2.x, point2.y

    # Convert latitude and longitude from degrees to radians
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)

    # Calculate the distance using pyproj's Geod class
    geod = pyproj.Geod(ellps='WGS84')
    azimuth1, azimuth2, distance = geod.inv(lon1_rad, lat1_rad, lon2_rad, lat2_rad)
    return distance

In [118]:
# session_starts = []

current_session_start = None
on_odometer = None
off_odometer = None

trip_ends = defaultdict(list)
calendar = defaultdict(list)
stops = defaultdict(list)
stop_times = defaultdict(list)
trips = defaultdict(list)
routes = defaultdict(list)
shapes = defaultdict(list)
feed_info = defaultdict(list)
agency = defaultdict(list)


df = df[df['Reason'].isin(['Arr. Way Point','Break End','Break Start','Dep. Way Point','Logoff','Logon','Status Report','Ignition Off','Ignition On','Odometer Offset'])]

agency["agency_id"].append("MGL")
agency["agency_name"].append("Microgrid Labs")
agency["agency_url"].append("https://www.microgridlabs.com")
agency["agency_timezone"].append("Asia/Kolkata")

calendar["start_date"].append(pd.to_datetime(df['GPS'].min()).strftime('%Y%m%d'))
calendar["end_date"].append(pd.to_datetime(df['GPS'].max()).strftime('%Y%m%d'))
calendar["monday"].append("1")
calendar["tuesday"].append("1")
calendar["wednesday"].append("1")
calendar["thursday"].append("1")
calendar["friday"].append("1")
calendar["saturday"].append("1")
calendar["sunday"].append("1")
calendar["service_id"].append("S1")

feed_info["feed_publisher_name"].append("Microgrid Labs")
feed_info["feed_publisher_url"].append("https://www.microgridlabs.com")
feed_info["feed_lang"].append("eng")
feed_info["feed_start_date"].append("20230101")
feed_info["feed_end_date"].append("20250101")
feed_info["feed_version"].append("1")
feed_info["feed_url"].append("https://www.microgridlabs.com")

trip_no = 1
block_id = 1
prev_start , prev_stop = None, None
for index, row in df.iterrows():
    if row['Reason'] == 'Ignition On':
        current_session_start = index
        on_odometer = row['Odometer']
    elif row['Reason'] == 'Ignition Off' and current_session_start is not None:
        off_odometer = row['Odometer']
        if on_odometer is not None and off_odometer is not None:
            diff = off_odometer - on_odometer
            if diff > 1:
                
                trips_df = df.loc[current_session_start:index].reset_index()
                   

                # Set Trips file here
                trips["trip_id"].append(f"T_{trip_no}")
                trips["route_id"].append(f"R_{trip_no}")
                trips["service_id"].append("S1")

                trips["shape_id"].append(f"Sh_{trip_no}")
                trips["trip_short_name"].append(f"Short Name {trip_no}")
                trips["direction_id"].append("1")

                # print(trips_df['Latitude'][0])
                # Set Block ID here
                bf_p = Point(depot[0],depot[1]).distance(Point(trips_df['Latitude'][0],trips_df['Longitude'][0]))
                # if Point(depot[0],depot[1]).buffer(1).contains(Point(trips_df['Latitude'][0],trips_df['Longitude'][0])):
                #     block_id+=1
                # print(bf_p)

                if check_buffer(Point(depot[0],depot[1]),Point(trips_df['Latitude'][0],trips_df['Longitude'][0])) <= 10 :
                    block_id+=1
                    
                trips["block_id"].append(f"B{block_id}")
                # Set Routes file here
                routes["agency_id"].append("MGL")
                routes["route_id"].append(f"R_{trip_no}")
                routes["route_short_name"].append(f"RSn {trip_no}")
                routes["route_long_name"].append(f"Long Name {trip_no}")
                routes["route_type"].append(3)
                
                prev_start = current_session_start
                prev_stop = index

                for id,trips_row in trips_df.iterrows():    
                    
                    stop_id = round(abs(float(trips_row['Latitude'])),3).__str__() + round(abs(float(trips_row['Longitude'])),3).__str__() + pd.to_datetime(trips_row['GPS']).strftime('%Y%m%d%H%M%S') + id.__str__()
                    
                    # Set Stops File - DONE
                    if id%5 == 0:
                        stops["stop_id"].append(f"{stop_id}")
                        stops["stop_name"].append(f"Stop {id}")
                        stops["stop_lat"].append(trips_row['Latitude'])
                        stops["stop_lon"].append(trips_row['Longitude'])

                        # Set Stop Times File - DONE
                        stop_times["stop_id"].append(f"{stop_id}")
                        stop_times["trip_id"].append(f"T_{trip_no}")
                        stop_times["timepoint"].append("0")
                        stop_times["stop_sequence"].append(f"{id}")
                        stop_times["arrival_time"].append(f"{trips_row['GPS'].strftime('%H:%M:%S')}")
                        stop_times["departure_time"].append(f"{trips_row['GPS'].strftime('%H:%M:%S')}")

                    # Set Shapes File - DONE
                    shapes["shape_id"].append(f"Sh_{trip_no}")
                    shapes["shape_pt_lat"].append(trips_row['Latitude']) 
                    shapes["shape_pt_lon"].append(trips_row['Longitude'])
                    shapes["shape_pt_sequence"].append(f"{id}")

                # Proceed to next Trip ID
                trip_no += 1
                

        else:
            # Do something when the Ignition On or Off row does not have any odometer value
            print("Ignition On or Off do not have an Odometer Value")

In [119]:
temp_dir = 'output'
pd.DataFrame(agency).to_csv(f"{temp_dir}/agency.txt",index=False)
pd.DataFrame(feed_info).to_csv(f"{temp_dir}/feed_info.txt",index=False)
pd.DataFrame(calendar).to_csv(f"{temp_dir}/calendar.txt",index=False,quoting=1)
pd.DataFrame(stops).to_csv(f"{temp_dir}/stops.txt",index=False,quoting=1)
pd.DataFrame(stop_times).to_csv(f"{temp_dir}/stop_times.txt",index=False,quoting=1)
pd.DataFrame(trips).to_csv(f"{temp_dir}/trips.txt",index=False,quoting=1)
pd.DataFrame(routes).to_csv(f"{temp_dir}/routes.txt",index=False,quoting=1)
pd.DataFrame(shapes).to_csv(f"{temp_dir}/shapes.txt",index=False,quoting=1)


with zipfile.ZipFile("gtfs_output.zip", "w") as zip_file:
    for filename in ["agency.txt","feed_info.txt","stop_times.txt", "stops.txt", "trips.txt", "routes.txt","calendar.txt","shapes.txt"]:
        zip_file.write(f"{temp_dir}/{filename}", arcname=filename)

In [81]:
# RUN EVERYTHING FROM HERE
